In [77]:
!pip install geopy
!pip install geopandas

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopy
import geopandas as gpd
from tqdm.notebook import tqdm
# pd.set_option('display.max_colwidth', -1)

from google.colab import drive
drive.mount('/content/drive')

patterns_paths = [
                  '/content/drive/My Drive/SafeGraph data/march_safegraph_updated.csv',
                  '/content/drive/My Drive/SafeGraph data/april_safegraph_updated.csv',
                  '/content/drive/My Drive/SafeGraph data/may_safegraph_updated.csv',
                  '/content/drive/My Drive/SafeGraph data/june_safegraph_updated.csv',
                  '/content/drive/My Drive/SafeGraph data/july_safegraph_updated.csv',
                  '/content/drive/My Drive/SafeGraph data/august_safegraph_updated.csv',
                  '/content/drive/My Drive/SafeGraph data/september_safegraph_updated.csv'
                  '/content/drive/My Drive/SafeGraph data/october_safegraph_updated.csv'
                 ]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
monthly_patterns_raw = pd.concat(map(pd.read_csv, patterns_paths)).reset_index(drop=True)
monthly_patterns_raw['poi_cbg'] = (monthly_patterns_raw['poi_cbg'] / 10).astype(int)

In [80]:
monthly_patterns_raw.head()

,safegraph_place_id,location_name,street_address,city,region,postal_code,safegraph_brand_ids,brands,date_range_start,date_range_end,raw_visit_counts,raw_visitor_counts,visits_by_day,poi_cbg,visitor_home_cbgs,visitor_daytime_cbgs,visitor_work_cbgs,visitor_country_of_origin,distance_from_home,median_dwell,bucketed_dwell_times,related_same_day_brand,related_same_month_brand,popularity_by_hour,popularity_by_day,device_type,placekey
0,sg:01d30e9d0c0d45dba33a963a443b41eb,Flako and Victor Auto Diagnostic Repair,10875 San Fernando Rd Ste 5,Pacoima,CA,91331,NaN,NaN,2020-03-01T00:00:00-08:00,2020-04-01T00:00:00-07:00,5,4,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",6037104404,{},"{""060371341012"":4}",{},{},NaN,6.0,"{""<5"":0,""5-20"":3,""21-60"":0,""61-240"":2,"">240"":0}","{""Food 4 Less"":100}","{""Smart & Final"":75,""ARCO"":74,""Domino's Pizza""...","[0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,3,0,0,1,1,0,0,0,0]","{""Monday"":1,""Tuesday"":0,""Wednesday"":0,""Thursda...","{""android"":0,""ios"":0}",NaN
1,sg:02e7de2c19284c6a930ade85565ae8bc,Baked,6027 Reseda Blvd,Tarzana,CA,91356,NaN,NaN,2020-03-01T00:00:00-08:00,2020-04-01T00:00:00-07:00,75,33,"[0, 6, 4, 5, 4, 3, 3, 1, 5, 2, 4, 1, 3, 0, 1, ...",6037139302,"{""060371393021"":4,""060371394012"":4,""0603712710...","{""060371393021"":6,""060371112063"":4,""0603713130...","{""060371393021"":4}","{""US"":28}",8705.0,19.0,"{""<5"":3,""5-20"":35,""21-60"":3,""61-240"":9,"">240"":25}","{""Vons"":33,""Restaurant Depot"":25}","{""Starbucks"":32,""McDonald's"":31,""Costco Wholes...","[0,1,1,1,1,1,3,4,11,14,26,31,32,34,31,25,14,3,...","{""Monday"":17,""Tuesday"":15,""Wednesday"":12,""Thur...","{""android"":17,""ios"":19}",NaN
2,sg:18d6ea1dc7444521a0d67c23af363d92,Men's Center of L A,21243 Ventura Blvd Ste 214,Woodland Hills,CA,91364,NaN,NaN,2020-03-01T00:00:00-08:00,2020-04-01T00:00:00-07:00,1,1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6037137502,{},{},{},{},NaN,31.0,"{""<5"":0,""5-20"":0,""21-60"":1,""61-240"":0,"">240"":0}","{""Jack in the Box"":100}","{""Fry's Electronics"":100,""Ralphs"":100,""99 Cent...","[0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]","{""Monday"":1,""Tuesday"":0,""Wednesday"":0,""Thursda...","{""android"":4,""ios"":0}",NaN
3,sg:1c5f590874264663b9c03c4d2d25694d,Molani Medical Group Jabeen Fatima M D,4955 Van Nuys Blvd Ste 415,Sherman Oaks,CA,91403,NaN,NaN,2020-03-01T00:00:00-08:00,2020-04-01T00:00:00-07:00,74,41,"[2, 8, 3, 3, 8, 4, 1, 0, 6, 3, 4, 4, 5, 1, 0, ...",6037128910,"{""060371394023"":4,""060371234101"":4,""0603713410...","{""060371412013"":4,""060371111001"":4,""0603713800...",{},"{""US"":40}",10611.0,63.5,"{""<5"":3,""5-20"":14,""21-60"":17,""61-240"":39,"">240...","{""Carl's Jr."":14,""Jack in the Box"":14}","{""Ralphs"":34,""Starbucks"":29,""In-N-Out Burger"":...","[0,0,0,0,0,0,0,7,22,26,21,20,15,14,19,18,5,2,1...","{""Monday"":20,""Tuesday"":12,""Wednesday"":10,""Thur...","{""android"":23,""ios"":16}",NaN
4,sg:2d57e61d4b304a4680c9da11929780bb,Smile Masters Dental Implants & Orthodontics G...,17703 Vanowen St,Reseda,CA,91335,NaN,NaN,2020-03-01T00:00:00-08:00,2020-04-01T00:00:00-07:00,13,13,"[0, 0, 1, 0, 2, 0, 0, 2, 1, 2, 0, 0, 1, 0, 0, ...",6037132300,"{""060371098001"":4,""060371320023"":4,""0603712160...","{""060371327003"":4,""060376013024"":4,""0603719161...","{""060376013021"":4}","{""US"":12}",5727.0,38.0,"{""<5"":1,""5-20"":4,""21-60"":5,""61-240"":3,"">240"":0}",{},"{""Ralphs"":31,""Chevron"":28,""Starbucks"":24,""Cost...","[1,0,0,0,0,0,0,0,0,1,3,4,2,1,0,0,1,0,2,2,0,1,1,2]","{""Monday"":1,""Tuesday"":5,""Wednesday"":1,""Thursda...","{""android"":4,""ios"":8}",NaN


In [83]:
import requests
requests.packages.urllib3.disable_warnings()

base_url = 'https://tea4avgrover.tea.texas.gov/public/rest/services/locator/GeocodeServer/findAddressCandidates?f=json&address='

for val in tqdm(monthly_patterns_raw['safegraph_place_id'].unique()):
  df = monthly_patterns_raw[monthly_patterns_raw['safegraph_place_id'] == val]
  row = df.iloc[0]
  if (row['Latitude'] > 0 or row['Longitude'] < 0):
    continue
  request_url = base_url + row['street_address'] + '&city=' + row['city'] + "&postal=" + str(row['postal_code'])
  try:
    response = requests.get(request_url, verify=False)
    response = response.json()
    if (len(response['candidates']) > 0):
      output = response['candidates'][0]
      latitude = output['location']['y']
      longitude = output['location']['x']
      for idx in df.index:
        monthly_patterns_raw.loc[idx, 'Latitude'] = latitude
        monthly_patterns_raw.loc[idx, 'Longitude'] = longitude
    else:
      print(request_url)
      for idx in df.index:
        monthly_patterns_raw.loc[idx, 'Latitude'] = 0
        monthly_patterns_raw.loc[idx, 'Longitude'] = 0
  except:
      print("BROKEN: ", request_url)
      for idx in df.index:
        monthly_patterns_raw.loc[idx, 'Latitude'] = 0
        monthly_patterns_raw.loc[idx, 'Longitude'] = 0



https://tea4avgrover.tea.texas.gov/public/rest/services/locator/GeocodeServer/findAddressCandidates?f=json&address=1123 Vine St # 14&city=Los Angeles&postal=90038
https://tea4avgrover.tea.texas.gov/public/rest/services/locator/GeocodeServer/findAddressCandidates?f=json&address=16461 Ventura Blvd # 1632&city=Encino&postal=91436
https://tea4avgrover.tea.texas.gov/public/rest/services/locator/GeocodeServer/findAddressCandidates?f=json&address=6115 Lankershim Blvd # 23&city=North Hollywood&postal=91606
https://tea4avgrover.tea.texas.gov/public/rest/services/locator/GeocodeServer/findAddressCandidates?f=json&address=21943 Ventura Blvd # 2194&city=Woodland Hills&postal=91364
https://tea4avgrover.tea.texas.gov/public/rest/services/locator/GeocodeServer/findAddressCandidates?f=json&address=2919 Los Feliz Blvd. #6 Franciscan Metro Center&city=Los Angeles&postal=90039
https://tea4avgrover.tea.texas.gov/public/rest/services/locator/GeocodeServer/findAddressCandidates?f=json&address=7902 Van Nuys 

In [84]:
march_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-03-01T00:00:00-08:00']
march_data.to_csv('march_safegraph_updated.csv', encoding='utf-8', index=False)

april_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-04-01T00:00:00-07:00']
april_data.to_csv('april_safegraph_updated.csv', encoding='utf-8', index=False)

may_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-05-01T00:00:00-07:00']
may_data.to_csv('may_safegraph_updated.csv', encoding='utf-8', index=False)

june_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-06-01T00:00:00-07:00']
june_data.to_csv('june_safegraph_updated.csv', encoding='utf-8', index=False)

july_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-07-01T00:00:00-07:00']
july_data.to_csv('july_safegraph_updated.csv', encoding='utf-8', index=False)

august_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-08-01T00:00:00-07:00']
august_data.to_csv('august_safegraph_updated.csv', encoding='utf-8', index=False)

september_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-09-01T00:00:00-07:00']
september_data.to_csv('september_safegraph_updated.csv', encoding='utf-8', index=False)

october_data = monthly_patterns_raw[monthly_patterns_raw['date_range_start'] == '2020-10-01T00:00:00-07:00']
october_data.to_csv('october_safegraph_updated.csv', encoding='utf-8', index=False)